In this notebook, am trying to develop a Recurent Neural Network from scratch,

This may sound weird and challenging , am doing it for learning purpose, I want to understand those architecture how they works and how we can use them especially for neural machine translation.

Theory : 
#### what is a Recurent neural network

Recurent neural network or RNN was created to make use of sequentials information.
So what are sequentials information, sequentials information are information that comes into a sequence, where the curent sequence depend on the previous sequence.

Another way of thinking about recurent neural network it's that they saved in memory what have been calculated so far.

They looks like this in the real world 

![RNN](./pictures/rnn.jpg)*RNN Achitecture [Source](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/)*

- $x_t$:  is the input of at time t, it can be a word in a sentence which is modeled as a one hot encoded vector.
- $s_t$ : is the hidden state of the network which is calculated from the previous hidden input and the input at current step $s_t = f(Ux_t+Ws_{t-1}))$
$f$ is usually a non linear function such as Relu or a tanh
- $o_t$ : is the output at time t , for word prediction $o_t$ is an array of all probabilities of word in our corpus.
$o_t=softmax(Vs_t)$

Some particularities of the network:
    
    the parameter W, U, V are shared accross all the network in oposition to the feed forward network
 We do not need always the output at each time , some times we cares only about the final output.

what a neural network can do:
    - with this we can predict time series data such as stock exchange prices
    - Do Neural Machine Translation, what I care about right now
    - Images tagging 
    - Speech recogniton, it may be useful to check this for local languages

#### Implementation

Let see how to implement this stuff now...

 the now see what are the major component of the arctecturere

what do we do to go from an imput $x_t $ to an output $o_t$?

- we calculate $s_t$, given $s_{t-1}$

Utilites functions are :

In [6]:
import numpy as np

In [7]:
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output
def sigmoid_output_to_derivative(output):
    return output*(1-output)
def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=0)

In [8]:
from collections import namedtuple
def get_params(vocab_size, num_hiddens):
    """
    function help us to return  initialize the neural networks parameters 
    for text prediction the the number of input and the number of output is equals to the vocabulary size, as well
    """
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return np.random.normal(scale=0.01, size=shape)
    # Hidden layer parameters
    U = normal((num_inputs, num_hiddens))
    W = normal((num_hiddens, num_hiddens))
    # Output layer parameters
    V = normal((num_hiddens, num_outputs))
    # Attach gradients
    Params = namedtuple('Params', ["U", "V", "W"])
    params = Params(U=U, V=V, W=W)
    return params

In [9]:
def compute_the_hidden_fuction(U, xt, W, s_prev):
    """
    this function calculate the value at st 
    Args:
         U is a matrix
         W is a matrix
         xt is the input at time t
         s_prev the value calculate in the previous layer
    """
    return np.tanh(np.dot(U, xt)+np.dot(W, s_prev))

In [10]:
def compute_ouput_ot(V, st):
    """
    this compute the output at time 
    Args:
        V : it's hyper parameter V which is a matrix
        st: the result from the hidden function
    """
    return softmax(np.dot(V, st))

the 2  functions by themselves will helps us to do the forward pass to the network, we just have to repeat the process n times where n is the number of hiddens units we have in our network

Let define a forward pass function:

In [31]:
def forward_pass(x, output_shape, vocab_size):
    """
    this function will perform a forward pass into our network 
    args :
        x : the input matrix we are using with it shape
        we fill them with zeroat tht begining
        output_shape : how we want our output to look like
    """
    n = x.shape[1] # number of element in the x  we are predicting the next sentence number of hiddens is n
    outputs = []
    params = get_params(vocab_size, num_hiddens=n)
    previous_s = np.zeros((n, 1))
    for i in range(1, n+1):
        st = compute_the_hidden_fuction(params.U, x[i, :, :], params.W, previous_s[i-1])
        previous_s.append(st)
        output_t = compute_ouput_ot(params.V, st)
        outputs.append(output_t)
        
    return previous_s, outputs

getting the data to use fr the training :

### II. Loading the data

I have already prepared the data in [this notebook](./text_processing.ipynb) I can pick up from where I left things and continue doing my stuff.

In [12]:
def load_document(path):
    """
    load the document at the given path
    """
    with open(path, 'r') as file:
        text = file.read()
        return text 

In [13]:
doc = load_document('data/republic_sentences.txt')
lines = doc.split('\n')

In [14]:
lines[:5]

['i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted',
 'i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted with',
 'went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted with the',
 'down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in wha

In [15]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

Using TensorFlow backend.
/Users/es.py/.pyenv/versions/3.6.5/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [16]:
sequences= np.array(sequences)

In [17]:
from keras.utils import to_categorical

Now we can one hot encode our variables 

In [18]:
X, Y = sequences[:, :-1], sequences[:, -1]

Checkings 

In [19]:
' '.join([tokenizer.index_word.get(index) for index in X[0]])

'i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was'

In [20]:
''.join([tokenizer.index_word.get(index) for index in Y][0])

'delighted'

In [21]:
vocab_size = len(tokenizer.word_index) + 1

In [22]:
Y = to_categorical(Y, num_classes=vocab_size)

In [23]:
X = to_categorical(X, num_classes=vocab_size)

Now we have our input and output set up, let now start with the serious stuffs 

In [24]:
X.shape

(118632, 50, 7410)

In [21]:
Y.shape

(118632, 7410)

let try forward pass

In [32]:
forward_pass(X, Y.shape[1], vocab_size=vocab_size)

ValueError: shapes (50,50) and (1,) not aligned: 50 (dim 1) != 1 (dim 0)

In [30]:
X[0, :, :]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)